# Import Required Libraries
Import libraries such as `scipy.io` for loading MATLAB files and `matplotlib` for visualization.

In [1]:
# Import necessary libraries
from scipy.io import loadmat  # For loading MATLAB files
import matplotlib.pyplot as plt  # For data visualization
import numpy as np  # For numerical operations

# Load MATLAB Files
Load the MATLAB files from the `P1` and `P3` directories using `scipy.io.loadmat`.

In [5]:
import os
import h5py
from scipy.io import loadmat

def load_mat_file(file_path):
    """
    Load a MATLAB file, handling both v7.3 (HDF5) and earlier formats.
    
    Args:
        file_path (str): Path to the MATLAB file.
    
    Returns:
        dict or h5py.File: Loaded data as a dictionary (for earlier formats) or HDF5 file object (for v7.3).
    """
    if not os.path.exists(file_path):
        print(f"Error: File not found - {file_path}")
        return None

    try:
        # Try loading as a non-v7.3 file
        data = loadmat(file_path)
        print(f"Loaded non-v7.3 file: {file_path}")
        return data
    except NotImplementedError:
        # If v7.3, use h5py
        try:
            with h5py.File(file_path, 'r') as f:
                print(f"Loaded v7.3 file: {file_path}")
                return {key: f[key] for key in f.keys()}
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")
            return None

# Example usage
data_dir = "../data"  # Base directory for your data
subdirs = ["P5"]  # Subdirectories to process

for subdir in subdirs:
    subdir_path = os.path.join(data_dir, subdir)
    if not os.path.exists(subdir_path):
        print(f"Error: Directory not found - {subdir_path}")
        continue

    for file_name in os.listdir(subdir_path):
        if file_name.endswith(".mat"):
            file_path = os.path.join(subdir_path, file_name)
            data = load_mat_file(file_path)
            if data is not None:
                print(f"Keys in {file_name}: {list(data.keys()) if isinstance(data, dict) else 'HDF5 keys'}")

Loaded non-v7.3 file: ../data/P5/135Deg_F1_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat
Keys in 135Deg_F1_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat: ['__header__', '__version__', '__globals__', 'MUPulses', 'Cost', 'PNR', 'SIG', 'IPTs', 'fsamp', 'startSIGInt', 'stopSIGInt', 'ref_signal', 'origRecMode', 'SIGFilePath', 'SIGFileName', 'DecompRuns', 'IED', 'discardChannelsVec', 'description', 'SIGlength', 'ProcTime', 'DecompStat']
Loaded non-v7.3 file: ../data/P5/90Deg_F2_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat
Keys in 90Deg_F2_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat: ['__header__', '__version__', '__globals__', 'MUPulses', 'Cost', 'PNR', 'SIG', 'IPTs', 'fsamp', 'startSIGInt', 'stopSIGInt', 'ref_signal', 'origRecMode', 'SIGFilePath', 'SIGFileName', 'DecompRuns', 'IED', 'discardChannelsVec', 'description', 'SIGlength', 'ProcTime', 'DecompStat']
Loaded non-v7.3 file: ../data/P5/90Deg_F1_1_plateau_iAll_MUe

In [9]:
# ————— Inspect a single file —————
file_path = "../data/P5/135Deg_F1_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat"
data_ret = load_mat_file(file_path)

# handle two return types (dict-only vs dict+h5py.File)
if isinstance(data_ret, tuple):
    data_dict, h5file = data_ret
else:
    data_dict = data_ret
    h5file = None

# Grab ref_signal and turn into a plain ndarray
ref_raw = data_dict['SIG']
if isinstance(ref_raw, h5py.Dataset):
    ref_arr = ref_raw[()]
elif h5file is not None and isinstance(ref_raw, h5py.Reference):
    ref_arr = h5file[ref_raw][()]
else:
    ref_arr = np.asarray(ref_raw)

# Squeeze out any singleton dims
ref_arr = np.squeeze(ref_arr)

# Now inspect
print(f"→ ref_arr type: {type(ref_arr)}, dtype: {ref_arr.dtype}")
print(f"→ ref_arr shape: {ref_arr.shape}")
print("→ first 10 samples:\n", ref_arr[:10])

# If you get shape (N,2), you’ve got [Fx, Fy] pairs;
# if it’s (N,) it’s still 1-D.
if ref_arr.ndim == 2 and ref_arr.shape[1] == 2:
    fx, fy = ref_arr[:,0], ref_arr[:,1]
    print(f"Detected two channels—Fx and Fy:\n  Fx[:5]={fx[:5]}\n  Fy[:5]={fy[:5]}")


Loaded non-v7.3 file: ../data/P5/135Deg_F1_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat
→ ref_arr type: <class 'numpy.ndarray'>, dtype: object
→ ref_arr shape: (8, 2)
→ first 10 samples:
 [[array([[0.0213623 , 0.04781087, 0.05950927, ..., 0.01475016, 0.02492269,
          0.02085368]], shape=(1, 529998), dtype=float32)
  array([[0.04374186, 0.05900065, 0.05645752, ..., 0.02034505, 0.02746582,
          0.03611246]], shape=(1, 529998), dtype=float32)                 ]
 [array([[-0.00050863,  0.04221598,  0.04882812, ...,  0.03204346,
           0.03407796,  0.03509521]], shape=(1, 529998), dtype=float32)
  array([[0.00305176, 0.02644857, 0.05137125, ..., 0.01373291, 0.00661214,
          0.01373291]], shape=(1, 529998), dtype=float32)                 ]
 [array([[0.01424154, 0.04170736, 0.05391439, ..., 0.02644857, 0.03306071,
          0.03407796]], shape=(1, 529998), dtype=float32)
  array([[0.02339681, 0.04933675, 0.05696614, ..., 0.02034505, 0.02339681,
          0

In [12]:
import os
import numpy as np
from scipy.io import loadmat

def load_mat_file(path):
    # exactly your existing loader
    from scipy.io import loadmat
    try:
        return loadmat(path), None
    except NotImplementedError:
        import h5py
        f = h5py.File(path, 'r')
        return {k: f[k] for k in f.keys()}, f

file_path = "../data/P5/135Deg_F1_1_plateau_iAll_MUedit.mat_decomp_120It_FDI_DEMUSEEDIT_cut_C.mat"
data_dict, h5file = load_mat_file(file_path)

def inspect_var(name, var):
    # unwrap h5py.Dataset
    if 'h5py' in str(type(var)):
        arr = var[()]
    else:
        arr = var

    # make it a numpy array where possible
    try:
        arr = np.asarray(arr)
    except:
        pass

    print(f"\n{name}:")
    print("  type =", type(var))
    print("  dtype=", getattr(arr, 'dtype', None))
    print("  shape=", getattr(arr, 'shape', None))

    # if it’s an object array, look at the first element
    if isinstance(arr, np.ndarray) and arr.dtype == object and arr.size > 0:
        elt = arr.flat[0]
        print("   → object-element type:", type(elt))
        try:
            sub = np.asarray(elt)
            print("     sub-shape:", sub.shape, "sub-dtype:", sub.dtype)
        except:
            pass

for key in data_dict:
    if key.startswith("__"): continue
    inspect_var(key, data_dict[key])

if h5file is not None:
    h5file.close()


MUPulses:
  type = <class 'numpy.ndarray'>
  dtype= object
  shape= (1, 10)
   → object-element type: <class 'numpy.ndarray'>
     sub-shape: (1, 330) sub-dtype: int32

Cost:
  type = <class 'numpy.ndarray'>
  dtype= object
  shape= (1, 10)
   → object-element type: <class 'numpy.ndarray'>
     sub-shape: (1, 1) sub-dtype: float64

PNR:
  type = <class 'numpy.ndarray'>
  dtype= float64
  shape= (1, 10)

SIG:
  type = <class 'numpy.ndarray'>
  dtype= object
  shape= (8, 2)
   → object-element type: <class 'numpy.ndarray'>
     sub-shape: (1, 529998) sub-dtype: float32

IPTs:
  type = <class 'numpy.ndarray'>
  dtype= float64
  shape= (10, 529998)

fsamp:
  type = <class 'numpy.ndarray'>
  dtype= uint16
  shape= (1, 1)

startSIGInt:
  type = <class 'numpy.ndarray'>
  dtype= uint8
  shape= (1, 1)

stopSIGInt:
  type = <class 'numpy.ndarray'>
  dtype= float64
  shape= (1, 1)

ref_signal:
  type = <class 'numpy.ndarray'>
  dtype= float32
  shape= (529998, 1)

origRecMode:
  type = <class 'n

In [13]:
import os
import numpy as np
from scipy.io import loadmat

def load_mat_file(path):
    # exactly your existing loader
    from scipy.io import loadmat
    try:
        return loadmat(path), None
    except NotImplementedError:
        import h5py
        f = h5py.File(path, 'r')
        return {k: f[k] for k in f.keys()}, f

file_path = "../data/P3/135Deg_F1_1_plateau_iAll_MUedit.mat_decomp_100It_FDI_DEMUSEEDIT_cut.mat"
data_dict, h5file = load_mat_file(file_path)

def inspect_var(name, var):
    # unwrap h5py.Dataset
    if 'h5py' in str(type(var)):
        arr = var[()]
    else:
        arr = var

    # make it a numpy array where possible
    try:
        arr = np.asarray(arr)
    except:
        pass

    print(f"\n{name}:")
    print("  type =", type(var))
    print("  dtype=", getattr(arr, 'dtype', None))
    print("  shape=", getattr(arr, 'shape', None))

    # if it’s an object array, look at the first element
    if isinstance(arr, np.ndarray) and arr.dtype == object and arr.size > 0:
        elt = arr.flat[0]
        print("   → object-element type:", type(elt))
        try:
            sub = np.asarray(elt)
            print("     sub-shape:", sub.shape, "sub-dtype:", sub.dtype)
        except:
            pass

for key in data_dict:
    if key.startswith("__"): continue
    inspect_var(key, data_dict[key])

if h5file is not None:
    h5file.close()


IED:
  type = <class 'numpy.ndarray'>
  dtype= uint8
  shape= (1, 1)

IPTs:
  type = <class 'numpy.ndarray'>
  dtype= float64
  shape= (18, 529998)

MUPulses:
  type = <class 'numpy.ndarray'>
  dtype= object
  shape= (1, 18)
   → object-element type: <class 'numpy.ndarray'>
     sub-shape: (1, 362) sub-dtype: int32

SIG:
  type = <class 'numpy.ndarray'>
  dtype= object
  shape= (8, 2)
   → object-element type: <class 'numpy.ndarray'>
     sub-shape: (1, 529998) sub-dtype: float32

SIGFileName:
  type = <class 'numpy.ndarray'>
  dtype= <U52
  shape= (1,)

SIGFilePath:
  type = <class 'numpy.ndarray'>
  dtype= <U108
  shape= (1,)

SIGlength:
  type = <class 'numpy.ndarray'>
  dtype= float64
  shape= (1, 1)

discardChannelsVec:
  type = <class 'numpy.ndarray'>
  dtype= uint8
  shape= (8, 2)

fsamp:
  type = <class 'numpy.ndarray'>
  dtype= uint16
  shape= (1, 1)

origRecMode:
  type = <class 'numpy.ndarray'>
  dtype= <U4
  shape= (1,)

ref_signal:
  type = <class 'numpy.ndarray'>
  dtype